In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from pybaseball import statcast, cache
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import joblib
import math
from pybaseball import statcast
import scipy.stats as stats
from catboost import Pool
import optuna
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score, precision_score
import sqlite3
cache.enable()

In [2]:
with sqlite3.connect("../../../Desktop/MLB Statcast.db") as conn:
    sc_23 = pd.read_sql_query("SELECT * FROM statcast_data_2023", conn)
    sc_22 = pd.read_sql_query("SELECT * FROM statcast_data_2022", conn)
    sc_21 = pd.read_sql_query("SELECT * FROM statcast_data_2021", conn)
    sc_20 = pd.read_sql_query("SELECT * FROM statcast_data_2020", conn)
conn.close()

In [3]:
total_sc = pd.concat([sc_23, sc_22, sc_21, sc_20])

In [4]:
[c for c in total_sc.columns]

['pitch_type',
 'game_date',
 'release_speed',
 'release_pos_x',
 'release_pos_z',
 'player_name',
 'batter',
 'pitcher',
 'events',
 'description',
 'spin_dir',
 'spin_rate_deprecated',
 'break_angle_deprecated',
 'break_length_deprecated',
 'zone',
 'des',
 'game_type',
 'stand',
 'p_throws',
 'home_team',
 'away_team',
 'type',
 'hit_location',
 'bb_type',
 'balls',
 'strikes',
 'game_year',
 'pfx_x',
 'pfx_z',
 'plate_x',
 'plate_z',
 'on_3b',
 'on_2b',
 'on_1b',
 'outs_when_up',
 'inning',
 'inning_topbot',
 'hc_x',
 'hc_y',
 'tfs_deprecated',
 'tfs_zulu_deprecated',
 'fielder_2',
 'umpire',
 'sv_id',
 'vx0',
 'vy0',
 'vz0',
 'ax',
 'ay',
 'az',
 'sz_top',
 'sz_bot',
 'hit_distance_sc',
 'launch_speed',
 'launch_angle',
 'effective_speed',
 'release_spin_rate',
 'release_extension',
 'game_pk',
 'pitcher.1',
 'fielder_2.1',
 'fielder_3',
 'fielder_4',
 'fielder_5',
 'fielder_6',
 'fielder_7',
 'fielder_8',
 'fielder_9',
 'release_pos_y',
 'estimated_ba_using_speedangle',
 'estimat

In [5]:
features = ['balls', 'strikes', 'plate_x', 'plate_z']
total_sc = total_sc[total_sc[features].notnull().all(axis=1)]
total_sc = total_sc[(total_sc['balls'] < 4) & (total_sc['strikes'] < 3)]

In [6]:
total_sc = total_sc[~total_sc['pitch_name'].isin(['Pitch Out', 'Eephus', 'Knuckleball'])]

In [7]:
fastballs = ['4-Seam Fastball', 'Sinker']
offspeed = ['Split-Finger', 'Changeup', 'Forkball']
breaking = ['Curveball', 'Slider', 'Cutter', 'Knuckle Curve', 'Sweeper', 'Screwball', 'Slow Curve']

In [8]:
total_sc['description'] = np.where(total_sc['description'] == 'hit_into_play', total_sc['events'], total_sc['description'])
field_outs = ['force_out', 'grounded_into_double_play', 'fielders_choice_out', 'fielders_choice', 'field_out', 'double_play', 'sac_fly', 'field_error', 'sac_fly_double_play', 'triple_play']
total_sc['description'] = total_sc['description'].replace(field_outs, 'field_out')

In [9]:
total_sc.description.unique()

array(['swinging_strike', 'foul', 'called_strike', 'field_out', 'ball',
       'single', 'foul_tip', 'swinging_strike_blocked', 'blocked_ball',
       'double', 'hit_by_pitch', 'sac_bunt', 'home_run', 'triple',
       'foul_bunt', 'missed_bunt', 'bunt_foul_tip', 'catcher_interf',
       'unknown_strike', 'sac_bunt_double_play'], dtype=object)

In [10]:
total_sc['whiff'] = total_sc['description'].isin(['swinging_strike', 'swinging_strike_blocked'])
total_sc['foul'] = total_sc['description'].isin(['foul', 'foul_tip'])
total_sc['in_play'] = total_sc['description'].isin(['single', 'double', 'triple', 'home_run', 'field_out'])
total_sc['swing'] = (total_sc['whiff'] | total_sc['foul'] | total_sc['in_play'])

total_sc['take'] = (~total_sc['swing'] & (total_sc['description'].isin(['hit_by_pitch', 'ball', 'called_strike', 'blocked_ball'])))
total_sc['hbp'] = total_sc['description'] == 'hit_by_pitch'
total_sc['ball'] = total_sc['description'].isin(['blocked_ball', 'ball'])
total_sc['strike'] = total_sc['description'] == 'called_strike'

total_sc['single'] = total_sc['description'] == 'single'
total_sc['double'] = total_sc['description'] == 'double'
total_sc['triple'] = total_sc['description'] == 'triple'
total_sc['home_run'] = total_sc['description'] == 'home_run'
total_sc['field_out'] = total_sc['description'] == 'field_out'

In [11]:
total_sc.loc[total_sc['swing'] & total_sc['foul'], 'type_swing'] = 'foul'
total_sc.loc[total_sc['swing'] & total_sc['in_play'], 'type_swing'] = 'in_play'
total_sc.loc[total_sc['swing'] & total_sc['whiff'], 'type_swing'] = 'whiff'

total_sc.loc[total_sc['take'] & total_sc['hbp'], 'type_take'] = 'hbp'
total_sc.loc[total_sc['take'] & total_sc['ball'], 'type_take'] = 'ball'
total_sc.loc[total_sc['take'] & total_sc['strike'], 'type_take'] = 'strike'

total_sc.loc[total_sc['in_play'] & total_sc['single'], 'type_in_play'] = 'single'
total_sc.loc[total_sc['in_play'] & total_sc['double'], 'type_in_play'] = 'double'
total_sc.loc[total_sc['in_play'] & total_sc['triple'], 'type_in_play'] = 'triple'
total_sc.loc[total_sc['in_play'] & total_sc['home_run'], 'type_in_play'] = 'home_run'
total_sc.loc[total_sc['in_play'] & total_sc['field_out'], 'type_in_play'] = 'field_out'

In [12]:
total_sc = total_sc.dropna(subset=['swing', 'take'])
total_sc = total_sc[total_sc['swing'] != total_sc['take']]

In [13]:
from sklearn.preprocessing import LabelEncoder
from hyperopt import hp, fmin, tpe

def objective(space, X_train, X_test, y_train, y_test):
    model = XGBClassifier(
        max_depth=int(space['max_depth']),
        gamma=space['gamma'],
        reg_alpha=int(space['reg_alpha']),
        reg_lambda=space['reg_lambda'],
        colsample_bytree=space['colsample_bytree'],
        min_child_weight=int(space['min_child_weight']),
        n_estimators=int(space['n_estimators']))
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return -accuracy

le_swing = LabelEncoder()

swing_df = total_sc[(total_sc['swing']) & total_sc['pitch_name'].isin(fastballs)]
swing_X = swing_df[features]
swing_y = le_swing.fit_transform(swing_df['type_swing'])

X_train, X_test, y_train, y_test = train_test_split(swing_X, swing_y, test_size=0.2, random_state=np.random.seed())

space = {
    'max_depth': hp.quniform("max_depth", 3, 18, 1),
    'gamma': hp.uniform('gamma', 1, 9),
    'reg_alpha': hp.quniform('reg_alpha', 40, 180, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'min_child_weight': hp.quniform('min_child_weight', 0, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 50, 200, 1),
    'seed': 12
}

best_params = fmin(fn=lambda params: objective(params, X_train, X_test, y_train, y_test),
                   space=space,
                   algo=tpe.suggest,
                   max_evals=10)
best_params = {
    'max_depth': int(best_params['max_depth']),
    'gamma': float(best_params['gamma']),
    'reg_alpha': int(best_params['reg_alpha']),
    'reg_lambda': float(best_params['reg_lambda']),
    'colsample_bytree': float(best_params['colsample_bytree']),
    'min_child_weight': int(best_params['min_child_weight']),
    'n_estimators': int(best_params['n_estimators'])
}
print("Best parameters:", best_params)

swing_model = XGBClassifier(**best_params)
swing_model.fit(X_train, y_train)

100%|██████████████████████████████████████████████| 10/10 [06:56<00:00, 41.66s/trial, best loss: -0.48639895226802116]
Best parameters: {'max_depth': 12, 'gamma': 2.960522259441297, 'reg_alpha': 105, 'reg_lambda': 0.5463174986770309, 'colsample_bytree': 0.8413875159847803, 'min_child_weight': 3, 'n_estimators': 177}


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.8413875159847803, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=2.960522259441297, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_threshold=64,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=12, max_leaves=0,
              min_child_weight=3, missing=nan, monotone_constraints='()',
              n_estimators=177, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', ...)

In [14]:
swing_df = total_sc[(total_sc['swing']) & total_sc['pitch_name'].isin(breaking)]
swing_X = swing_df[features]
swing_y = le_swing.fit_transform(swing_df['type_swing'])

X_train, X_test, y_train, y_test = train_test_split(swing_X, swing_y, test_size=0.2, random_state=np.random.seed())

best_params = fmin(fn=lambda params: objective(params, X_train, X_test, y_train, y_test),
                   space=space,
                   algo=tpe.suggest,
                   max_evals=10)
best_params = {
    'max_depth': int(best_params['max_depth']),
    'gamma': float(best_params['gamma']),
    'reg_alpha': int(best_params['reg_alpha']),
    'reg_lambda': float(best_params['reg_lambda']),
    'colsample_bytree': float(best_params['colsample_bytree']),
    'min_child_weight': int(best_params['min_child_weight']),
    'n_estimators': int(best_params['n_estimators'])
}
print(best_params)

swing_bb_model = XGBClassifier(**best_params)
swing_bb_model.fit(X_train, y_train)

100%|███████████████████████████████████████████████| 10/10 [05:02<00:00, 30.22s/trial, best loss: -0.5013923632416104]
{'max_depth': 16, 'gamma': 8.312282992758863, 'reg_alpha': 42, 'reg_lambda': 0.1942717870070988, 'colsample_bytree': 0.976312880247004, 'min_child_weight': 10, 'n_estimators': 126}


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.976312880247004, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=8.312282992758863, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_threshold=64,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=16, max_leaves=0,
              min_child_weight=10, missing=nan, monotone_constraints='()',
              n_estimators=126, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', ...)

In [15]:
swing_df = total_sc[(total_sc['swing']) & total_sc['pitch_name'].isin(offspeed)]
swing_X = swing_df[features]
swing_y = le_swing.fit_transform(swing_df['type_swing'])

X_train, X_test, y_train, y_test = train_test_split(swing_X, swing_y, test_size=0.2, random_state=np.random.seed())
best_params = fmin(fn=lambda params: objective(params, X_train, X_test, y_train, y_test),
                   space=space,
                   algo=tpe.suggest,
                   max_evals=10)
best_params = {
    'max_depth': int(best_params['max_depth']),
    'gamma': float(best_params['gamma']),
    'reg_alpha': int(best_params['reg_alpha']),
    'reg_lambda': float(best_params['reg_lambda']),
    'colsample_bytree': float(best_params['colsample_bytree']),
    'min_child_weight': int(best_params['min_child_weight']),
    'n_estimators': int(best_params['n_estimators'])
}
print(best_params)

swing_offs_model = XGBClassifier(**best_params)
swing_offs_model.fit(X_train, y_train)

100%|███████████████████████████████████████████████| 10/10 [02:06<00:00, 12.69s/trial, best loss: -0.4686764159127525]
{'max_depth': 6, 'gamma': 5.30781746409081, 'reg_alpha': 78, 'reg_lambda': 0.6413586805189868, 'colsample_bytree': 0.6853445306288419, 'min_child_weight': 2, 'n_estimators': 179}


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.6853445306288419, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=5.30781746409081, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_threshold=64,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=6, max_leaves=0,
              min_child_weight=2, missing=nan, monotone_constraints='()',
              n_estimators=179, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', ...)

In [16]:
le_take = LabelEncoder()

take_total_sc = total_sc[(total_sc['take']) & total_sc['pitch_name'].isin(fastballs)]
take_X = take_total_sc[features]
take_y = le_take.fit_transform(take_total_sc['type_take'])

X_train, X_test, y_train, y_test = train_test_split(take_X, take_y, test_size=0.2, random_state=np.random.seed())
best_params = fmin(fn=lambda params: objective(params, X_train, X_test, y_train, y_test),
                   space=space,
                   algo=tpe.suggest,
                   max_evals=10)
best_params = {
    'max_depth': int(best_params['max_depth']),
    'gamma': float(best_params['gamma']),
    'reg_alpha': int(best_params['reg_alpha']),
    'reg_lambda': float(best_params['reg_lambda']),
    'colsample_bytree': float(best_params['colsample_bytree']),
    'min_child_weight': int(best_params['min_child_weight']),
    'n_estimators': int(best_params['n_estimators'])
}
print(best_params)

take_model = XGBClassifier(**best_params)
take_model.fit(X_train, y_train)

100%|███████████████████████████████████████████████| 10/10 [06:38<00:00, 39.84s/trial, best loss: -0.9170444171624642]
{'max_depth': 4, 'gamma': 8.642609762772157, 'reg_alpha': 134, 'reg_lambda': 0.5517524410413727, 'colsample_bytree': 0.5752827422086888, 'min_child_weight': 4, 'n_estimators': 149}


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.5752827422086888, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=8.642609762772157, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_threshold=64,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=4, max_leaves=0,
              min_child_weight=4, missing=nan, monotone_constraints='()',
              n_estimators=149, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', ...)

In [17]:
take_total_sc = total_sc[(total_sc['take']) & total_sc['pitch_name'].isin(breaking)]
take_X = take_total_sc[features]
take_y = le_take.fit_transform(take_total_sc['type_take'])

X_train, X_test, y_train, y_test = train_test_split(take_X, take_y, test_size=0.2, random_state=np.random.seed())
best_params = fmin(fn=lambda params: objective(params, X_train, X_test, y_train, y_test),
                   space=space,
                   algo=tpe.suggest,
                   max_evals=10)
best_params = {
    'max_depth': int(best_params['max_depth']),
    'gamma': float(best_params['gamma']),
    'reg_alpha': int(best_params['reg_alpha']),
    'reg_lambda': float(best_params['reg_lambda']),
    'colsample_bytree': float(best_params['colsample_bytree']),
    'min_child_weight': int(best_params['min_child_weight']),
    'n_estimators': int(best_params['n_estimators'])
}
print(best_params)

take_bb_model = XGBClassifier(**best_params)
take_bb_model.fit(X_train, y_train)

100%|███████████████████████████████████████████████| 10/10 [04:46<00:00, 28.60s/trial, best loss: -0.9324187252179793]
{'max_depth': 16, 'gamma': 1.628851537533242, 'reg_alpha': 121, 'reg_lambda': 0.1523793026836182, 'colsample_bytree': 0.854157140251582, 'min_child_weight': 9, 'n_estimators': 189}


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.854157140251582, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1.628851537533242, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_threshold=64,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=16, max_leaves=0,
              min_child_weight=9, missing=nan, monotone_constraints='()',
              n_estimators=189, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', ...)

In [18]:
take_total_sc = total_sc[(total_sc['take']) & total_sc['pitch_name'].isin(offspeed)]
take_X = take_total_sc[features]
take_y = le_take.fit_transform(take_total_sc['type_take'])

X_train, X_test, y_train, y_test = train_test_split(take_X, take_y, test_size=0.2, random_state=np.random.seed())
best_params = fmin(fn=lambda params: objective(params, X_train, X_test, y_train, y_test),
                   space=space,
                   algo=tpe.suggest,
                   max_evals=10)
best_params = {
    'max_depth': int(best_params['max_depth']),
    'gamma': float(best_params['gamma']),
    'reg_alpha': int(best_params['reg_alpha']),
    'reg_lambda': float(best_params['reg_lambda']),
    'colsample_bytree': float(best_params['colsample_bytree']),
    'min_child_weight': int(best_params['min_child_weight']),
    'n_estimators': int(best_params['n_estimators'])
}
print(best_params)

take_offs_model = XGBClassifier(**best_params)
take_offs_model.fit(X_train, y_train)

100%|███████████████████████████████████████████████| 10/10 [01:34<00:00,  9.48s/trial, best loss: -0.9459103749357987]
{'max_depth': 9, 'gamma': 4.743504316948279, 'reg_alpha': 58, 'reg_lambda': 0.008411542679003237, 'colsample_bytree': 0.6218802648910653, 'min_child_weight': 6, 'n_estimators': 82}


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.6218802648910653, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=4.743504316948279, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_threshold=64,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=9, max_leaves=0,
              min_child_weight=6, missing=nan, monotone_constraints='()',
              n_estimators=82, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', ...)

In [19]:
le_woba = LabelEncoder()

woba_total_sc = total_sc[(total_sc['in_play']) & total_sc['pitch_name'].isin(fastballs)]
woba_X = woba_total_sc[features]
woba_y = le_woba.fit_transform(woba_total_sc['type_in_play'])

X_train, X_test, y_train, y_test = train_test_split(woba_X, woba_y, test_size=0.2, random_state=np.random.seed())
best_params = fmin(fn=lambda params: objective(params, X_train, X_test, y_train, y_test),
                   space=space,
                   algo=tpe.suggest,
                   max_evals=10)
best_params = {
    'max_depth': int(best_params['max_depth']),
    'gamma': float(best_params['gamma']),
    'reg_alpha': int(best_params['reg_alpha']),
    'reg_lambda': float(best_params['reg_lambda']),
    'colsample_bytree': float(best_params['colsample_bytree']),
    'min_child_weight': int(best_params['min_child_weight']),
    'n_estimators': int(best_params['n_estimators'])
}
print(best_params)

woba_model = XGBClassifier(**best_params)
woba_model.fit(X_train, y_train)

100%|███████████████████████████████████████████████| 10/10 [03:27<00:00, 20.71s/trial, best loss: -0.6645271715875053]
{'max_depth': 11, 'gamma': 6.850710348858795, 'reg_alpha': 107, 'reg_lambda': 0.7321923274979867, 'colsample_bytree': 0.7447974845990599, 'min_child_weight': 1, 'n_estimators': 66}


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.7447974845990599, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=6.850710348858795, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_threshold=64,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=11, max_leaves=0,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=66, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', ...)

In [20]:
woba_total_sc = total_sc[(total_sc['in_play']) & total_sc['pitch_name'].isin(breaking)]
woba_X = woba_total_sc[features]
woba_y = le_woba.fit_transform(woba_total_sc['type_in_play'])

X_train, X_test, y_train, y_test = train_test_split(woba_X, woba_y, test_size=0.2, random_state=np.random.seed())
best_params = fmin(fn=lambda params: objective(params, X_train, X_test, y_train, y_test),
                   space=space,
                   algo=tpe.suggest,
                   max_evals=10)
best_params = {
    'max_depth': int(best_params['max_depth']),
    'gamma': float(best_params['gamma']),
    'reg_alpha': int(best_params['reg_alpha']),
    'reg_lambda': float(best_params['reg_lambda']),
    'colsample_bytree': float(best_params['colsample_bytree']),
    'min_child_weight': int(best_params['min_child_weight']),
    'n_estimators': int(best_params['n_estimators'])
}
print(best_params)

woba_bb_model = XGBClassifier(**best_params)
woba_bb_model.fit(X_train, y_train)

100%|███████████████████████████████████████████████| 10/10 [02:15<00:00, 13.51s/trial, best loss: -0.6860267066357815]
{'max_depth': 13, 'gamma': 8.736714389279326, 'reg_alpha': 108, 'reg_lambda': 0.7644354058487527, 'colsample_bytree': 0.8836105442683017, 'min_child_weight': 8, 'n_estimators': 118}


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.8836105442683017, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=8.736714389279326, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_threshold=64,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=13, max_leaves=0,
              min_child_weight=8, missing=nan, monotone_constraints='()',
              n_estimators=118, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', ...)

In [21]:
woba_total_sc = total_sc[(total_sc['in_play']) & total_sc['pitch_name'].isin(offspeed)]
woba_X = woba_total_sc[features]
woba_y = le_woba.fit_transform(woba_total_sc['type_in_play'])

X_train, X_test, y_train, y_test = train_test_split(woba_X, woba_y, test_size=0.2, random_state=np.random.seed())
best_params = fmin(fn=lambda params: objective(params, X_train, X_test, y_train, y_test),
                   space=space,
                   algo=tpe.suggest,
                   max_evals=10)
best_params = {
    'max_depth': int(best_params['max_depth']),
    'gamma': float(best_params['gamma']),
    'reg_alpha': int(best_params['reg_alpha']),
    'reg_lambda': float(best_params['reg_lambda']),
    'colsample_bytree': float(best_params['colsample_bytree']),
    'min_child_weight': int(best_params['min_child_weight']),
    'n_estimators': int(best_params['n_estimators'])
}
print(best_params)

woba_offs_model = XGBClassifier(**best_params)
woba_offs_model.fit(X_train, y_train)

100%|███████████████████████████████████████████████| 10/10 [00:48<00:00,  4.85s/trial, best loss: -0.6872894585490099]
{'max_depth': 10, 'gamma': 2.7749622141134394, 'reg_alpha': 153, 'reg_lambda': 0.017008569186843014, 'colsample_bytree': 0.6690832948029344, 'min_child_weight': 10, 'n_estimators': 172}


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.6690832948029344, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=2.7749622141134394, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_threshold=64,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=10, max_leaves=0,
              min_child_weight=10, missing=nan, monotone_constraints='()',
              n_estimators=172, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', ...)

In [22]:
def will_swing_objective(trial, will_swing_X, will_swing_y):
    X_train, X_test, y_train, y_test = train_test_split(will_swing_X, will_swing_y, test_size=0.2, random_state=np.random.seed())
    
    params = {
        "iterations": trial.suggest_int("iterations", 1000, 2000),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", .05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100)
    }

    model = CatBoostClassifier(**params, silent=True, thread_count=-1)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:, 1]  # Probability of class 1 (swing)
    logloss = log_loss(y_test, y_pred)
    return logloss

will_swing_total_sc = total_sc[total_sc['pitch_name'].isin(fastballs)]
will_swing_X = will_swing_total_sc[features]
will_swing_y = will_swing_total_sc['swing']

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: will_swing_objective(trial, will_swing_X, will_swing_y), n_trials=6)
best_params = study.best_params
best_ll = study.best_value

print('Best hyperparameters:', best_params)
print('Best logloss:', best_ll)
X_train, X_test, y_train, y_test = train_test_split(will_swing_X, will_swing_y, test_size=0.2, random_state=np.random.seed())

will_swing_model = CatBoostClassifier(**best_params, silent=True)
will_swing_model.fit(X_train, y_train)

[I 2023-12-02 14:30:56,562] A new study created in memory with name: no-name-4af9b0d8-b67f-4732-9f76-e71990a358ed
[I 2023-12-02 14:32:42,550] Trial 0 finished with value: 0.5535480407692867 and parameters: {'iterations': 1679, 'learning_rate': 0.005512198872413826, 'depth': 6, 'colsample_bylevel': 0.12213977426252609, 'min_data_in_leaf': 88}. Best is trial 0 with value: 0.5535480407692867.
[I 2023-12-02 14:35:36,276] Trial 1 finished with value: 0.47461367220227685 and parameters: {'iterations': 1163, 'learning_rate': 0.00426939126324989, 'depth': 7, 'colsample_bylevel': 0.986522437154733, 'min_data_in_leaf': 26}. Best is trial 1 with value: 0.47461367220227685.
[I 2023-12-02 14:38:42,069] Trial 2 finished with value: 0.4710335958249565 and parameters: {'iterations': 1499, 'learning_rate': 0.08063595376927422, 'depth': 6, 'colsample_bylevel': 0.4896087324460142, 'min_data_in_leaf': 61}. Best is trial 2 with value: 0.4710335958249565.
[I 2023-12-02 14:41:22,008] Trial 3 finished with va

Best hyperparameters: {'iterations': 1499, 'learning_rate': 0.08063595376927422, 'depth': 6, 'colsample_bylevel': 0.4896087324460142, 'min_data_in_leaf': 61}
Best logloss: 0.4710335958249565


In [23]:
will_swing_total_sc = total_sc[total_sc['pitch_name'].isin(breaking)]
will_swing_X = will_swing_total_sc[features]
will_swing_y = will_swing_total_sc['swing']

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: will_swing_objective(trial, will_swing_X, will_swing_y), n_trials=6)
best_params = study.best_params
best_ll = study.best_value

print('Best hyperparameters:', best_params)
print('Best logloss:', best_ll)
X_train, X_test, y_train, y_test = train_test_split(will_swing_X, will_swing_y, test_size=0.2, random_state=np.random.seed())

will_swing_bb_model = CatBoostClassifier(**best_params, silent=True)
will_swing_bb_model.fit(X_train, y_train)

[I 2023-12-02 14:49:47,161] A new study created in memory with name: no-name-919bf327-d0a4-42fb-8a3d-a5d4aa82fa22
[I 2023-12-02 14:54:47,744] Trial 0 finished with value: 0.5101000563215158 and parameters: {'iterations': 1983, 'learning_rate': 0.018222182344479237, 'depth': 10, 'colsample_bylevel': 0.7107415700069754, 'min_data_in_leaf': 18}. Best is trial 0 with value: 0.5101000563215158.
[I 2023-12-02 14:56:21,982] Trial 1 finished with value: 0.5102838445759872 and parameters: {'iterations': 1101, 'learning_rate': 0.07521431063098293, 'depth': 3, 'colsample_bylevel': 0.9761718442856424, 'min_data_in_leaf': 91}. Best is trial 0 with value: 0.5101000563215158.
[I 2023-12-02 14:58:11,435] Trial 2 finished with value: 0.5457045846004622 and parameters: {'iterations': 1654, 'learning_rate': 0.0037972419169667346, 'depth': 8, 'colsample_bylevel': 0.21080929184102365, 'min_data_in_leaf': 75}. Best is trial 0 with value: 0.5101000563215158.
[I 2023-12-02 14:59:56,916] Trial 3 finished with 

Best hyperparameters: {'iterations': 1983, 'learning_rate': 0.018222182344479237, 'depth': 10, 'colsample_bylevel': 0.7107415700069754, 'min_data_in_leaf': 18}
Best logloss: 0.5101000563215158


In [24]:
will_swing_total_sc = total_sc[total_sc['pitch_name'].isin(offspeed)]
will_swing_X = will_swing_total_sc[features]
will_swing_y = will_swing_total_sc['swing']

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: will_swing_objective(trial, will_swing_X, will_swing_y), n_trials=6)
best_params = study.best_params
best_ll = study.best_value

print('Best hyperparameters:', best_params)
print('Best logloss:', best_ll)
X_train, X_test, y_train, y_test = train_test_split(will_swing_X, will_swing_y, test_size=0.2, random_state=np.random.seed())

will_swing_offs_model = CatBoostClassifier(**best_params, silent=True)
will_swing_offs_model.fit(X_train, y_train)

[I 2023-12-02 15:08:33,835] A new study created in memory with name: no-name-dee038eb-22b0-4789-97bb-531b03cc64fc
[I 2023-12-02 15:09:15,393] Trial 0 finished with value: 0.5231449938665349 and parameters: {'iterations': 1560, 'learning_rate': 0.00165714542798646, 'depth': 3, 'colsample_bylevel': 0.5970220197762633, 'min_data_in_leaf': 15}. Best is trial 0 with value: 0.5231449938665349.
[I 2023-12-02 15:09:41,022] Trial 1 finished with value: 0.5942872661897112 and parameters: {'iterations': 1047, 'learning_rate': 0.00110675511461775, 'depth': 2, 'colsample_bylevel': 0.9894686560277622, 'min_data_in_leaf': 8}. Best is trial 0 with value: 0.5231449938665349.
[I 2023-12-02 15:10:18,457] Trial 2 finished with value: 0.47338612428642646 and parameters: {'iterations': 1724, 'learning_rate': 0.09213803291190308, 'depth': 3, 'colsample_bylevel': 0.24959754610144114, 'min_data_in_leaf': 56}. Best is trial 2 with value: 0.47338612428642646.
[I 2023-12-02 15:10:40,912] Trial 3 finished with val

Best hyperparameters: {'iterations': 1736, 'learning_rate': 0.039945843690563046, 'depth': 4, 'colsample_bylevel': 0.8491883870852024, 'min_data_in_leaf': 55}
Best logloss: 0.47232952397179817


In [70]:
with sqlite3.connect("../../../Desktop/MLB Statcast.db") as conn:
    sc_23 = pd.read_sql_query("SELECT * FROM statcast_data_2023", conn)
conn.close()

In [71]:
sc_23 = sc_23[sc_23[features].notnull().all(axis=1)]
sc_23 = sc_23[(sc_23['balls'] < 4) & (sc_23['strikes'] < 3)]

In [72]:
fastball_df = sc_23[sc_23['pitch_name'].isin(fastballs)]
bb_df = sc_23[sc_23['pitch_name'].isin(breaking)]
offs_df = sc_23[sc_23['pitch_name'].isin(offspeed)]

In [73]:
values = {
    'home_run': 1.374328827219,
    'triple': 1.05755624961515,
    'double': 0.766083123898271,
    'single': 0.467292970729251,
    'ball': 0.0636883289483747,
    'hit_by_pitch': 0.0636883289483747,
    'blocked_ball': 0.0636883289483747,
    'foul': -0.0380502742575014,
    'foul_tip': -0.0380502742575014,
    'bunt_foul': -0.0380502742575014,
    'bunt_foul_tip': -0.0380502742575014,
    'called_strike': -0.065092516089806,
    'swinging_strike': -0.118124935770601,
    'swinging_strike_blocked': -0.118124935770601,
    'force_out': -0.1955687665555,
    'grounded_into_double_play': -0.1955687665555,
    'fielders_choice_out': -0.1955687665555,
    'fielders_choice': -0.1955687665555,
    'field_out': -0.1955687665555,
    'double_play': -0.1955687665555,
    'sac_fly': -0.236889645519856,
    'field_error': -0.236889645519856,
    'catcher_interf': -0.789788814378052,
    'sac_fly_double_play': -0.789788814378052,
    'triple_play': -0.789788814378052
}

In [74]:
swing_probs = swing_model.predict_proba(fastball_df[features])
take_probs = take_model.predict_proba(fastball_df[features])
ws_probs = will_swing_model.predict_proba(fastball_df[features])
woba_probs = woba_model.predict_proba(fastball_df[features])

fastball_df['whiff_prob'] = swing_probs[:, list(le_swing.inverse_transform(swing_model.classes_)).index('whiff')]
fastball_df['in_play_prob'] = swing_probs[:, list(le_swing.inverse_transform(swing_model.classes_)).index('in_play')]
fastball_df['foul_prob'] = swing_probs[:, list(le_swing.inverse_transform(swing_model.classes_)).index('foul')]

fastball_df['strike_prob'] = take_probs[:, list(le_take.inverse_transform(take_model.classes_)).index('strike')]
fastball_df['ball_prob'] = take_probs[:, list(le_take.inverse_transform(take_model.classes_)).index('ball')]
fastball_df['hbp_prob'] = take_probs[:, list(le_take.inverse_transform(take_model.classes_)).index('hbp')]

fastball_df['single_prob'] = woba_probs[:, list(le_woba.inverse_transform(woba_model.classes_)).index('single')]
fastball_df['double_prob'] = woba_probs[:, list(le_woba.inverse_transform(woba_model.classes_)).index('double')]
fastball_df['triple_prob'] = woba_probs[:, list(le_woba.inverse_transform(woba_model.classes_)).index('triple')]
fastball_df['hr_prob'] = woba_probs[:, list(le_woba.inverse_transform(woba_model.classes_)).index('home_run')]
fastball_df['fo_prob'] = woba_probs[:, list(le_woba.inverse_transform(woba_model.classes_)).index('field_out')]
fastball_df['xwOBAcon'] = values['single'] * fastball_df['single_prob'] + values['double'] * fastball_df['double_prob'] + values['triple'] * fastball_df['triple_prob'] + values['home_run'] * fastball_df['hr_prob'] + values['field_out'] * fastball_df['fo_prob']

fastball_df['swing_prob'] = ws_probs[:, list(will_swing_model.classes_).index('True')]
fastball_df['take_prob'] = 1 - fastball_df['swing_prob'] 

fastball_df['val_swing'] = values['swinging_strike'] * fastball_df['whiff_prob'] + values['foul'] * fastball_df['foul_prob']
+ fastball_df['xwOBAcon'] * fastball_df['in_play_prob']
fastball_df['val_take'] = values['called_strike'] * fastball_df['strike_prob'] + values['ball'] * fastball_df['ball_prob'] + values['hit_by_pitch'] * fastball_df['hbp_prob']
fastball_df['xRV'] = fastball_df['val_swing'] * fastball_df['swing_prob'] + fastball_df['val_take'] * fastball_df['take_prob']

C:\Users\kellyjc\AppData\Local\Temp\ipykernel_11312\3806814061.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fastball_df['whiff_prob'] = swing_probs[:, list(le_swing.inverse_transform(swing_model.classes_)).index('whiff')]
C:\Users\kellyjc\AppData\Local\Temp\ipykernel_11312\3806814061.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fastball_df['in_play_prob'] = swing_probs[:, list(le_swing.inverse_transform(swing_model.classes_)).index('in_play')]
C:\Users\kellyjc\AppData\Local\Temp\ipykernel_1131

In [75]:
swing_bb_probs = swing_bb_model.predict_proba(bb_df[features])
take_bb_probs = take_bb_model.predict_proba(bb_df[features])
ws_bb_probs = will_swing_bb_model.predict_proba(bb_df[features])
woba_bb_probs = woba_bb_model.predict_proba(bb_df[features])

bb_df['whiff_prob'] = swing_bb_probs[:, list(le_swing.inverse_transform(swing_bb_model.classes_)).index('whiff')]
bb_df['in_play_prob'] = swing_bb_probs[:, list(le_swing.inverse_transform(swing_bb_model.classes_)).index('in_play')]
bb_df['foul_prob'] = swing_bb_probs[:, list(le_swing.inverse_transform(swing_bb_model.classes_)).index('foul')]

bb_df['strike_prob'] = take_bb_probs[:, list(le_take.inverse_transform(take_bb_model.classes_)).index('strike')]
bb_df['ball_prob'] = take_bb_probs[:, list(le_take.inverse_transform(take_bb_model.classes_)).index('ball')]
bb_df['hbp_prob'] = take_bb_probs[:, list(le_take.inverse_transform(take_bb_model.classes_)).index('hbp')]

bb_df['single_prob'] = woba_bb_probs[:, list(le_woba.inverse_transform(woba_bb_model.classes_)).index('single')]
bb_df['double_prob'] = woba_bb_probs[:, list(le_woba.inverse_transform(woba_bb_model.classes_)).index('double')]
bb_df['triple_prob'] = woba_bb_probs[:, list(le_woba.inverse_transform(woba_bb_model.classes_)).index('triple')]
bb_df['hr_prob'] = woba_bb_probs[:, list(le_woba.inverse_transform(woba_bb_model.classes_)).index('home_run')]
bb_df['fo_prob'] = woba_bb_probs[:, list(le_woba.inverse_transform(woba_bb_model.classes_)).index('field_out')]
bb_df['xwOBAcon'] = values['single'] * bb_df['single_prob'] + values['double'] * bb_df['double_prob'] + values['triple'] * bb_df['triple_prob'] + values['home_run'] * bb_df['hr_prob'] + values['field_out'] * bb_df['fo_prob']

bb_df['swing_prob'] = ws_bb_probs[:, list(will_swing_bb_model.classes_).index('True')]
bb_df['take_prob'] = 1 - bb_df['swing_prob'] 

bb_df['val_swing'] = values['swinging_strike'] * bb_df['whiff_prob'] + values['foul'] * bb_df['foul_prob']
+ bb_df['xwOBAcon'] * bb_df['in_play_prob']
bb_df['val_take'] = values['called_strike'] * bb_df['strike_prob'] + values['ball'] * bb_df['ball_prob'] + values['hit_by_pitch'] * bb_df['hbp_prob']
bb_df['xRV'] = bb_df['val_swing'] * bb_df['swing_prob'] + bb_df['val_take'] * bb_df['take_prob']

C:\Users\kellyjc\AppData\Local\Temp\ipykernel_11312\1539665531.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['whiff_prob'] = swing_bb_probs[:, list(le_swing.inverse_transform(swing_bb_model.classes_)).index('whiff')]
C:\Users\kellyjc\AppData\Local\Temp\ipykernel_11312\1539665531.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['in_play_prob'] = swing_bb_probs[:, list(le_swing.inverse_transform(swing_bb_model.classes_)).index('in_play')]
C:\Users\kellyjc\AppData\Local\Temp\ipykernel_1131

In [76]:
swing_offs_probs = swing_offs_model.predict_proba(offs_df[features])
take_offs_probs = take_offs_model.predict_proba(offs_df[features])
ws_offs_probs = will_swing_offs_model.predict_proba(offs_df[features])
woba_offs_probs = woba_offs_model.predict_proba(offs_df[features])

offs_df['whiff_prob'] = swing_offs_probs[:, list(le_swing.inverse_transform(swing_offs_model.classes_)).index('whiff')]
offs_df['in_play_prob'] = swing_offs_probs[:, list(le_swing.inverse_transform(swing_offs_model.classes_)).index('in_play')]
offs_df['foul_prob'] = swing_offs_probs[:, list(le_swing.inverse_transform(swing_offs_model.classes_)).index('foul')]

offs_df['strike_prob'] = take_offs_probs[:, list(le_take.inverse_transform(take_offs_model.classes_)).index('strike')]
offs_df['ball_prob'] = take_offs_probs[:, list(le_take.inverse_transform(take_offs_model.classes_)).index('ball')]
offs_df['hbp_prob'] = take_offs_probs[:, list(le_take.inverse_transform(take_offs_model.classes_)).index('hbp')]

offs_df['single_prob'] = woba_offs_probs[:, list(le_woba.inverse_transform(woba_offs_model.classes_)).index('single')]
offs_df['double_prob'] = woba_offs_probs[:, list(le_woba.inverse_transform(woba_offs_model.classes_)).index('double')]
offs_df['triple_prob'] = woba_offs_probs[:, list(le_woba.inverse_transform(woba_offs_model.classes_)).index('triple')]
offs_df['hr_prob'] = woba_offs_probs[:, list(le_woba.inverse_transform(woba_offs_model.classes_)).index('home_run')]
offs_df['fo_prob'] = woba_offs_probs[:, list(le_woba.inverse_transform(woba_offs_model.classes_)).index('field_out')]
offs_df['xwOBAcon'] = values['single'] * offs_df['single_prob'] + values['double'] * offs_df['double_prob'] + values['triple'] * offs_df['triple_prob'] + values['home_run'] * offs_df['hr_prob'] + values['field_out'] * offs_df['fo_prob']

offs_df['swing_prob'] = ws_offs_probs[:, list(will_swing_offs_model.classes_).index('True')]
offs_df['take_prob'] = 1 - offs_df['swing_prob'] 

offs_df['val_swing'] = values['swinging_strike'] * offs_df['whiff_prob'] + values['foul'] * offs_df['foul_prob']
+ offs_df['xwOBAcon'] * offs_df['in_play_prob']
offs_df['val_take'] = values['called_strike'] * offs_df['strike_prob'] + values['ball'] * offs_df['ball_prob'] + values['hit_by_pitch'] * offs_df['hbp_prob']
offs_df['xRV'] = offs_df['val_swing'] * offs_df['swing_prob'] + offs_df['val_take'] * offs_df['take_prob']

C:\Users\kellyjc\AppData\Local\Temp\ipykernel_11312\4010475334.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offs_df['whiff_prob'] = swing_offs_probs[:, list(le_swing.inverse_transform(swing_offs_model.classes_)).index('whiff')]
C:\Users\kellyjc\AppData\Local\Temp\ipykernel_11312\4010475334.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offs_df['in_play_prob'] = swing_offs_probs[:, list(le_swing.inverse_transform(swing_offs_model.classes_)).index('in_play')]
C:\Users\kellyjc\AppData\Local\Temp\ip

In [77]:
total_df = pd.concat([fastball_df, bb_df, offs_df])

In [78]:
def calculate_location_plus(pitch_sc, min_num_pitches=100):
    agg_location_plus = pitch_sc.groupby(['player_name', 'pitch_name']).agg(
        mean_xrv=('xRV', 'mean'),
        mean_rv=('delta_run_exp', 'mean'),
        num_pitches=('player_name', 'count'),
        velo=('release_speed', 'mean'),
        spin_rate=('release_spin_rate', 'mean'),
        vert_break=('pfx_z', 'mean'),
        horz_break=('pfx_x', 'mean')
    ).reset_index()
    
    agg_location_plus['xrv_100'] = agg_location_plus['mean_xrv'] * 100
    agg_location_plus['xrv_100_adj'] = abs(agg_location_plus['xrv_100'] - agg_location_plus['xrv_100'].max())
    agg_location_plus['location_plus'] = (agg_location_plus['xrv_100_adj'] / agg_location_plus['xrv_100_adj'].mean()) * 100
    agg_location_plus = agg_location_plus.sort_values('location_plus', ascending=False)

    agg_location_plus = agg_location_plus[agg_location_plus['num_pitches'] > min_num_pitches]

    return agg_location_plus

def calculate_agg_location_plus(pitch_sc, min_num_pitches=100):
    agg_location_plus = pitch_sc.groupby(['player_name']).agg(
        mean_xrv=('xRV', 'mean'),  
        swing_prob=('swing_prob', 'mean'),
        whiff_prob=('whiff_prob', 'mean'),
        take_prob=('take_prob', 'mean'),
        xwobacon=('xwOBAcon', 'mean'),
        fo_prob=('fo_prob', 'mean'),
        mean_rv=('delta_run_exp', 'mean'),
        num_pitches=('player_name', 'count'),
        velo=('release_speed', 'mean'),
        spin_rate=('release_spin_rate', 'mean'),
        vert_break=('pfx_z', 'mean'),
        horz_break=('pfx_x', 'mean'),
        RV=('delta_run_exp', 'mean')
    ).reset_index()

    agg_location_plus['xrv_100'] = agg_location_plus['mean_xrv'] * 100
    agg_location_plus['xrv_100_adj'] = abs(agg_location_plus['xrv_100'] - agg_location_plus['xrv_100'].max())
    agg_location_plus['location_plus'] = (agg_location_plus['xrv_100_adj'] / agg_location_plus['xrv_100_adj'].mean()) * 100
    agg_location_plus = agg_location_plus.sort_values('location_plus', ascending=False)

    agg_location_plus = agg_location_plus[agg_location_plus['num_pitches'] > min_num_pitches]

    return agg_location_plus

agg_total = calculate_agg_location_plus(total_df, 1300)
agg_fb = calculate_location_plus(fastball_df, 300)
agg_bb = calculate_location_plus(bb_df, 300)
agg_offs = calculate_location_plus(offs_df, 300)

In [79]:
agg_total

,player_name,mean_xrv,swing_prob,whiff_prob,take_prob,xwobacon,fo_prob,mean_rv,num_pitches,velo,spin_rate,vert_break,horz_break,RV,xrv_100,xrv_100_adj,location_plus
380,"Kirby, George",-0.015722,0.534334,0.289768,0.465666,0.079770,0.680980,-0.008004,2824,91.609242,2155.348275,0.534965,-0.363293,-0.008004,-1.572185,7.678399,111.346285
479,"Mikolas, Miles",-0.015489,0.521932,0.300535,0.478068,0.076794,0.681526,0.002691,3287,87.508640,2280.620037,0.407295,-0.263806,0.002691,-1.548947,7.655161,111.009309
710,"Syndergaard, Noah",-0.014441,0.515677,0.294446,0.484323,0.074423,0.684544,0.011694,1404,88.278348,2028.245189,0.747835,-0.535883,0.011694,-1.444145,7.550360,109.489560
196,"Eflin, Zach",-0.014332,0.498431,0.313371,0.501569,0.074136,0.682909,-0.006092,2652,87.012971,2240.321631,0.279219,-0.186923,-0.006092,-1.433242,7.539457,109.331451
624,"Ryan, Joe",-0.013959,0.544858,0.312079,0.455142,0.076463,0.684361,-0.000295,2709,88.125323,2046.593333,0.764120,-0.615552,-0.000295,-1.395891,7.502105,108.789809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,"Senga, Kodai",-0.004595,0.441985,0.377558,0.558015,0.071250,0.687232,-0.010137,2795,89.383506,2094.372577,0.720182,-0.299277,-0.010137,-0.459457,6.565672,95.210365
728,"Toussaint, Touki",-0.003920,0.447248,0.337875,0.552752,0.071945,0.684666,0.000020,1595,87.063699,2170.259912,0.168050,-0.396075,0.000020,-0.391969,6.498184,94.231705
386,"Kopech, Michael",-0.003163,0.441586,0.337732,0.558414,0.077677,0.682588,0.005663,2430,91.280527,2400.334847,0.904955,-0.407128,0.005663,-0.316273,6.422487,93.134012
672,"Snell, Blake",-0.003043,0.434637,0.393891,0.565363,0.072792,0.684981,-0.013531,3261,90.137351,2296.413571,0.848675,0.146661,-0.013531,-0.304297,6.410511,92.960343


In [80]:
joblib.dump(woba_model, 'location_woba_fb.joblib')
joblib.dump(take_model, 'location_take_fb.joblib')
joblib.dump(will_swing_model, 'location_ws_fb.joblib')
joblib.dump(swing_model, 'location_swing_fb.joblib')

joblib.dump(woba_bb_model, 'location_woba_bb.joblib')
joblib.dump(take_bb_model, 'location_take_bb.joblib')
joblib.dump(will_swing_bb_model, 'location_ws_bb.joblib')
joblib.dump(swing_bb_model, 'location_swing_bb.joblib')

joblib.dump(woba_offs_model, 'location_woba_offs.joblib')
joblib.dump(take_offs_model, 'location_take_offs.joblib')
joblib.dump(will_swing_offs_model, 'location_ws_offs.joblib')
joblib.dump(swing_offs_model, 'location_swing_offs.joblib')

['location_swing_offs.joblib']

In [81]:
from scipy.stats import spearmanr
from pybaseball import pitching_stats_bref

data = pitching_stats_bref(2023)

agg_total[['Last Name', 'First Name']] = agg_total['player_name'].str.split(', ', expand=True)
agg_total['Full Name'] = agg_total['First Name'] + ' ' + agg_total['Last Name']
stats_location = data.merge(agg_total, left_on='Name', right_on='Full Name', how='inner')

sp_correlation_df = pd.DataFrame(index=['ERA', 'WHIP', 'SO9', 'RV', 'BAbip'])

for column in sp_correlation_df.index:
    corr, _ = spearmanr(stats_location['location_plus'], stats_location[column])
    sp_correlation_df.loc[column, 'Spearman Rank Correlation'] = corr
sp_correlation_df

,Spearman Rank Correlation
ERA,-0.090233
WHIP,-0.343089
SO9,-0.071684
RV,-0.091374
BAbip,-0.025745


In [82]:
from scipy.stats import pearsonr

data = pitching_stats_bref(2023)

agg_total[['Last Name', 'First Name']] = agg_total['player_name'].str.split(', ', expand=True)
agg_total['Full Name'] = agg_total['First Name'] + ' ' + agg_total['Last Name']
stats_location = data.merge(agg_total, left_on='Name', right_on='Full Name', how='inner')

p_correlation_df = pd.DataFrame(index=['ERA', 'WHIP', 'SO9', 'RV', 'BAbip'])

for column in p_correlation_df.index:
    corr, _ = pearsonr(stats_location['location_plus'], stats_location[column])
    p_correlation_df.loc[column, 'Pearson Correlation'] = corr
p_correlation_df

,Pearson Correlation
ERA,-0.080638
WHIP,-0.348398
SO9,-0.148035
RV,-0.068062
BAbip,-0.073880


In [83]:
sp_correlation_df.to_csv('correlations/spearman_sameyr_location.csv')
p_correlation_df.to_csv('correlations/pearson_sameyr_location.csv')